# PII Inference Notebook

In [1]:
INFERENCE_MAX_LENGTH=1024
STRIDE=384
model_path = 'model_dir/DeBERTA_1024_Last'

In [2]:
import json
import argparse
from itertools import chain
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
import numpy as np

## Tokenization

In [3]:
def tokenize(example, tokenizer):
    text = []
    token_map = []
    
    for idx, (t, ws) in enumerate(zip(example["tokens"], example["trailing_whitespace"])):
        text.append(t)
        token_map.extend([idx]*len(t))
        
        if ws:
            text.append(" ")
            token_map.append(-1)
        
        
    tokenized = tokenizer("".join(text), 
                          return_offsets_mapping=True, 
                          truncation=True, 
                          max_length=INFERENCE_MAX_LENGTH,
                          stride=STRIDE, 
                          return_overflowing_tokens=True)
        
    return {**tokenized, "token_map": token_map}

In [4]:
data = json.load(open("Dataset/competition/test.json"))
len(data)

10

In [5]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [x["document"] for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})

tokenizer = AutoTokenizer.from_pretrained(model_path)
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer}, num_proc=2)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

## Inferencing

In [6]:
model = AutoModelForTokenClassification.from_pretrained(model_path)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer = Trainer(
    model=model, 
    args=args, 
    data_collator=collator, 
    tokenizer=tokenizer,
)

## Ensemble

Originally from: https://www.kaggle.com/code/wonbulzhang/0-962-piidd-let-s-go-higher

In [7]:
import gc
import torch
import numpy as np
import time

from scipy.special import softmax

In [8]:
model_paths = {
    "model_dir/DeBERTA_1024_First": 1 / 3,
    "model_dir/DeBERTA_1024_Last": 1 / 3,
    "model_dir/DeBERTA_1024_Middle": 1 / 3,
}

In [9]:
def get_predictions(ds):
    all_preds = []

    for model_path, weight in model_paths.items():
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForTokenClassification.from_pretrained(model_path)
        collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
        args = TrainingArguments(
            ".",
            per_device_eval_batch_size=1,
            report_to="none",
        )
        trainer = Trainer(
            model=model,
            args=args,
            data_collator=collator,
            tokenizer=tokenizer,
        )
        predictions = trainer.predict(ds).predictions

        # This idea from this notebook: https://www.kaggle.com/code/olyatsimboy/912-blending-0-903-0-854-deberta3base
        weighted_predictions = softmax(predictions, axis=-1) * weight

        all_preds.append(weighted_predictions)
        del model, trainer
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(2)

    total_weight = sum(model_paths.values())
    weighted_average_predictions = np.sum(all_preds, axis=0) / total_weight
    return weighted_average_predictions

## Postprocessing

### Striding functions

As using the stride give an overlap in tokens, these have to be removed (either pick one side of the stride or average them, ...).

https://www.kaggle.com/code/valentinwerner/945-deberta-3-base-striding-inference/notebook

In [10]:
def backwards_map_preds(i, sub_predictions, max_len):
    # nothing to map backwards if sequence is too short to be split in the first place
    if max_len != 1:
        if i == 0:
            # First sequence needs no SEP token (used to end a sequence)
            sub_predictions = sub_predictions[:, :-1, :]
        elif i == max_len - 1:
            # End sequence needs to CLS token + Stride tokens
            sub_predictions = sub_predictions[
                :, 1 + STRIDE :, :
            ]  # CLS tokens + Stride tokens
        else:
            # Middle sequence needs to CLS token + Stride tokens + SEP token
            sub_predictions = sub_predictions[:, 1 + STRIDE : -1, :]

    return sub_predictions


def backwards_map_(i, row_attribute, max_len):
    # Same logics as for backwards_map_preds - except lists instead of 3darray
    if max_len != 1:
        if i == 0:
            row_attribute = row_attribute[:-1]
        elif i == max_len - 1:
            row_attribute = row_attribute[1 + STRIDE :]
        else:
            row_attribute = row_attribute[1 + STRIDE : -1]

    return row_attribute

In [11]:
# model_path = "model_dir/DeBERTA_1024_Last"
# model = AutoModelForTokenClassification.from_pretrained(model_path)
# collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
# args = TrainingArguments(
#     ".", 
#     per_device_eval_batch_size=1, 
#     report_to="none",
# )
# trainer = Trainer(
#     model=model, 
#     args=args, 
#     data_collator=collator, 
#     tokenizer=tokenizer,
# )

In [12]:
preds = []
ds_dict = {"document": [], "token_map": [], "offset_mapping": [], "tokens": []}

for row in ds:
    # keys that need to be re-assembled
    row_preds = []
    row_offset = []

    for i, y in enumerate(row["offset_mapping"]):
        # create new datasset for each of of the splits per document
        x = Dataset.from_dict(
            {
                "token_type_ids": [row["token_type_ids"][i]],
                "input_ids": [row["input_ids"][i]],
                "attention_mask": [row["attention_mask"][i]],
                "offset_mapping": [row["offset_mapping"][i]],
            }
        )

        # predict for that split
        pred = get_predictions(x)
        # pred = trainer.predict(x).predictions

        # removing the stride and additional CLS & SEP that are created
        row_preds.append(backwards_map_preds(i, pred, len(row["offset_mapping"])))
        row_offset += backwards_map_(i, y, len(row["offset_mapping"]))

    # Finalize row
    ds_dict["document"].append(row["document"])
    ds_dict["tokens"].append(row["tokens"])
    ds_dict["token_map"].append(row["token_map"])
    ds_dict["offset_mapping"].append(row_offset)

    # Finalize prediction collection by concattenating
    p_concat = np.concatenate(row_preds, axis=1)
    preds.append(p_concat)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
config = json.load(open(Path(model_path) / "config.json"))
id2label = config["id2label"]

preds_final = []
for p in preds:
    predictions = p.argmax(-1)
    predictions_without_O = p[:,:,:12].argmax(-1)
    O_predictions = p[:,:,12]

    threshold = 0.95
    preds_final.append(np.where(O_predictions < threshold, predictions_without_O , predictions))

## Reassembling

In [14]:
ds = Dataset.from_dict(ds_dict)
pairs = set()
document, token, label, token_str = [], [], [], []

for p, token_map, offsets, tokens, doc in zip(
    preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]
):
    for token_pred, (start_idx, end_idx) in zip(p[0], offsets):
        label_pred = id2label[str(token_pred)]

        if start_idx + end_idx == 0:
            continue

        if token_map[start_idx] == -1:
            start_idx += 1

        # ignore "\n\n"
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map):
            break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred != "O" and token_id != -1:
            pair = (doc, token_id)

            if pair not in pairs:
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                pairs.add(pair)

In [15]:
df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label,
    "token_str": token_str
})
df["row_id"] = list(range(len(df)))
display(df.head(100))

,document,token,label,token_str,row_id
0,7,9,B-NAME_STUDENT,Nathalie,0
1,7,10,I-NAME_STUDENT,Sylla,1
2,7,482,B-NAME_STUDENT,Nathalie,2
3,7,483,I-NAME_STUDENT,Sylla,3
4,7,741,B-NAME_STUDENT,Nathalie,4
5,7,742,I-NAME_STUDENT,Sylla,5
6,10,0,B-NAME_STUDENT,Diego,6
7,10,1,I-NAME_STUDENT,Estrada,7
8,10,464,B-NAME_STUDENT,Diego,8
9,10,465,I-NAME_STUDENT,Estrada,9


In [16]:
# df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)